In [1]:
# Setup tests
import pandas as pd
import re
import xml.etree.ElementTree as ET
import os
from datetime import datetime

# Configuration de test
TEST_CASES = [
    # Format: (query, expected_confidence_level, should_generate, expected_missing_entities)
    ("3x 10 min tempo avec 5min récup", "high", True, []),
    ("1h endurance aerobic", "medium", True, []),
    ("entraînement pyramide", "low", False, ["duration", "repetitions", "recovery"]),
    ("faire du vélo", "very_low", False, ["duration", "repetitions", "intensity", "recovery"]),
    ("5x30s sprint max", "medium", True, []),
]

print("🧪 Configuration des tests chargée")
print(f"   {len(TEST_CASES)} cas de test définis")


🧪 Configuration des tests chargée
   5 cas de test définis


In [2]:
# Importation des classes du pipeline (copie simplifiée pour les tests)
class ValidationConfig:
    CONFIDENCE_THRESHOLD_HIGH = 0.95    # Génération directe - EXCELLENCE REQUISE
    CONFIDENCE_THRESHOLD_MEDIUM = 0.75  # Génération avec avertissement - TRÈS BONNE CORRESPONDANCE
    CONFIDENCE_THRESHOLD_LOW = 0.75     # Rejet en dessous - QUALITÉ MAXIMALE EXIGÉE
    
    REQUIRED_ENTITIES = {
        'duration': r'\d+\s*(min|h|s|sec|secondes?|minutes?|heures?)',
        'repetitions': r'\d+\s*[x*×]\s*',
        'intensity': r'(VO2|seuil|tempo|aerobic|sprint|endurance|threshold|anaerobic|neuromuscular)',
        'recovery': r'(recup|rec|recovery|repos)\s*\d+'
    }

class TestValidator:
    def __init__(self):
        # Corpus complet pour tests (version simplifiée)
        self.corpus = [
            # Aerobic/Endurance
            {'description': '1h endurance allure aerobic tranquille', 'zone': 'Aerobic', 'variations': ['1h endur aerobic', '60min endurance facile']},
            {'description': '2h endurance longue aerobic base', 'zone': 'Aerobic', 'variations': ['2h endur longue', '120min aerobic']},
            {'description': '45min recuperation active facile', 'zone': 'Recovery', 'variations': ['45min recup active', 'recuperation 45min']},
            
            # Tempo/Sweet Spot
            {'description': '3x 10 min tempo sweet spot avec 5min recup', 'zone': 'Tempo', 'variations': ['3x10min tempo 5minrec', '3*10min sweet spot']},
            {'description': '4x 12 min tempo 88% CP avec 4min recup', 'zone': 'Tempo', 'variations': ['4x12min tempo 88%', '4*12min 88% CP']},
            {'description': '2x 20 min tempo sweet spot avec 10min recup', 'zone': 'Tempo', 'variations': ['2x20min tempo', '2*20min sweet spot']},
            
            # Threshold/Seuil
            {'description': '4x 8 min seuil threshold 95% CP avec 4min recup', 'zone': 'Threshold', 'variations': ['4x8min seuil 95%', '4*8min threshold']},
            {'description': '3x 12 min threshold FTP avec 6min recup', 'zone': 'Threshold', 'variations': ['3x12min FTP', '3*12min threshold']},
            
            # VO2max
            {'description': '5x 5 min VO2max 110% CP avec 5min recup', 'zone': 'VO2max', 'variations': ['5x5min VO2 110%', '5*5min VO2max']},
            {'description': '8x 3 min VO2max 115% CP avec 3min recup', 'zone': 'VO2max', 'variations': ['8x3min VO2 115%', '8*3min VO2max']},
            
            # Anaerobic
            {'description': '6x 1 min sprint anaerobic avec 2min recup', 'zone': 'Anaerobic', 'variations': ['6x1min sprint 2minrec', '6*1min anaerobic']},
            {'description': '10x 30s anaerobic avec 90s recup', 'zone': 'Anaerobic', 'variations': ['10x30s anaerobic 90s', '10*30sec sprint']},
            
            # Neuromuscular
            {'description': '5x 30s neuromuscular power max avec 4min30 recup complete', 'zone': 'Neuromuscular', 'variations': ['5x30s neuro max', '5*30sec sprint max']},
            {'description': '8x 15s sprint neuromuscular avec 3min recup', 'zone': 'Neuromuscular', 'variations': ['8x15s neuro sprint', '8*15sec neuromuscular']},
            
            # Complexes
            {'description': 'pyramide 1-2-3-4-3-2-1min VO2max avec 1min recup', 'zone': 'VO2max', 'variations': ['pyramide 1234321 VO2', 'pyramid 1-2-3-4-3-2-1min']},
            {'description': 'pyramide tempo 5-10-15-10-5min avec 5min recup', 'zone': 'Tempo', 'variations': ['pyramide tempo 5-10-15', 'pyramid tempo']},
            
            # Tests
            {'description': 'test 20min FTP maximal', 'zone': 'Test', 'variations': ['test FTP 20min', 'test 20min maximal']},
        ]
    
    def search_corpus(self, query):
        """Recherche sémantique dans le corpus de test"""
        query_lower = query.lower()
        best_match = None
        best_score = 0.0
        
        for workout in self.corpus:
            score = self._calculate_similarity(query_lower, workout['description'].lower())
            
            # Vérifier aussi les variations
            if 'variations' in workout:
                for variation in workout['variations']:
                    var_score = self._calculate_similarity(query_lower, variation.lower())
                    if var_score > score:
                        score = var_score * 1.1
            
            if score > best_score:
                best_score = score
                best_match = workout
        
        return best_match, min(best_score, 0.95)
    
    def _calculate_similarity(self, query, description):
        """Calcul de similarité simplifié pour tests"""
        score = 0.0
        query_words = set(query.split())
        desc_words = set(description.split())
        
        # Mots-clés avec poids
        keywords = {
            'aerobic': 0.3, 'endurance': 0.3, 'tempo': 0.3, 'sweet': 0.2, 'spot': 0.2,
            'threshold': 0.3, 'seuil': 0.3, 'ftp': 0.3, 'vo2max': 0.4, 'vo2': 0.4,
            'anaerobic': 0.3, 'neuromuscular': 0.4, 'neuro': 0.3, 'sprint': 0.3,
            'pyramide': 0.4, 'pyramid': 0.4, 'test': 0.3, 'recup': 0.2, 'recovery': 0.2
        }
        
        # Score basé sur mots-clés
        for word in query_words:
            if word in desc_words:
                weight = keywords.get(word, 0.1)
                score += weight
        
        # Bonus pour patterns numériques
        import re
        query_numbers = re.findall(r'\d+', query)
        desc_numbers = re.findall(r'\d+', description)
        
        for num in query_numbers:
            if num in desc_numbers:
                score += 0.15
        
        return max(0.0, score)
    
    def detect_missing_entities(self, query):
        entities = {}
        query_lower = query.lower()
        
        for entity_type, pattern in ValidationConfig.REQUIRED_ENTITIES.items():
            matches = re.findall(pattern, query_lower, re.IGNORECASE)
            entities[entity_type] = len(matches) > 0
        
        return [entity for entity, found in entities.items() if not found]
    
    def validate_query(self, query):
        best_match, confidence = self.search_corpus(query)
        
        # Fallback si aucun match trouvé
        if best_match is None:
            best_match = self.corpus[0]  # Premier entraînement par défaut
            confidence = 0.1
        
        missing_entities = self.detect_missing_entities(query)
        
        if confidence >= ValidationConfig.CONFIDENCE_THRESHOLD_HIGH:
            confidence_level = "high"
            is_valid = True
        elif confidence >= ValidationConfig.CONFIDENCE_THRESHOLD_MEDIUM:
            confidence_level = "medium"
            is_valid = True
        elif confidence >= ValidationConfig.CONFIDENCE_THRESHOLD_LOW:
            confidence_level = "low"
            is_valid = False
        else:
            confidence_level = "very_low"
            is_valid = False
        
        return {
            'confidence': confidence,
            'confidence_level': confidence_level,
            'is_valid': is_valid,
            'missing_entities': missing_entities,
            'corpus_match': best_match
        }

test_validator = TestValidator()
print("✅ Validateur de test initialisé")


✅ Validateur de test initialisé


In [3]:
# Test 1: Validation des seuils de confiance
def test_confidence_thresholds():
    """Teste que les seuils de confiance fonctionnent correctement"""
    print("🧪 TEST 1: Seuils de confiance")
    print("-" * 40)
    
    results = []
    for query, expected_level, should_generate, expected_missing in TEST_CASES:
        validation = test_validator.validate_query(query)
        
        # Test du niveau de confiance
        level_match = validation['confidence_level'] == expected_level
        
        # Test de la décision de génération
        generation_match = validation['is_valid'] == should_generate
        
        # Test des entités manquantes
        missing_match = set(validation['missing_entities']) == set(expected_missing)
        
        test_passed = level_match and generation_match and missing_match
        
        results.append({
            'query': query,
            'expected_level': expected_level,
            'actual_level': validation['confidence_level'],
            'expected_generate': should_generate,
            'actual_generate': validation['is_valid'],
            'expected_missing': expected_missing,
            'actual_missing': validation['missing_entities'],
            'passed': test_passed
        })
        
        status = "✅ PASS" if test_passed else "❌ FAIL"
        print(f"{status} | {query[:30]:<30} | {validation['confidence']:.3f} | {validation['confidence_level']}")
    
    passed_count = sum(1 for r in results if r['passed'])
    total_count = len(results)
    
    print(f"\n📊 Résultat: {passed_count}/{total_count} tests passés")
    
    if passed_count == total_count:
        print("✅ Tous les tests de seuils de confiance sont OK")
    else:
        print("❌ Certains tests ont échoué")
        for r in results:
            if not r['passed']:
                print(f"   ÉCHEC: {r['query']}")
                print(f"      Niveau attendu: {r['expected_level']}, obtenu: {r['actual_level']}")
                print(f"      Génération attendue: {r['expected_generate']}, obtenue: {r['actual_generate']}")
                print(f"      Entités manquantes attendues: {r['expected_missing']}, obtenues: {r['actual_missing']}")
    
    return results

test_confidence_thresholds()


🧪 TEST 1: Seuils de confiance
----------------------------------------
❌ FAIL | 3x 10 min tempo avec 5min récu | 0.950 | high
❌ FAIL | 1h endurance aerobic           | 0.850 | medium
❌ FAIL | entraînement pyramide          | 0.400 | very_low
✅ PASS | faire du vélo                  | 0.100 | very_low
❌ FAIL | 5x30s sprint max               | 0.770 | medium

📊 Résultat: 1/5 tests passés
❌ Certains tests ont échoué
   ÉCHEC: 3x 10 min tempo avec 5min récup
      Niveau attendu: high, obtenu: high
      Génération attendue: True, obtenue: True
      Entités manquantes attendues: [], obtenues: ['recovery']
   ÉCHEC: 1h endurance aerobic
      Niveau attendu: medium, obtenu: medium
      Génération attendue: True, obtenue: True
      Entités manquantes attendues: [], obtenues: ['repetitions', 'recovery']
   ÉCHEC: entraînement pyramide
      Niveau attendu: low, obtenu: very_low
      Génération attendue: False, obtenue: False
      Entités manquantes attendues: ['duration', 'repetitions', '

[{'query': '3x 10 min tempo avec 5min récup',
  'expected_level': 'high',
  'actual_level': 'high',
  'expected_generate': True,
  'actual_generate': True,
  'expected_missing': [],
  'actual_missing': ['recovery'],
  'passed': False},
 {'query': '1h endurance aerobic',
  'expected_level': 'medium',
  'actual_level': 'medium',
  'expected_generate': True,
  'actual_generate': True,
  'expected_missing': [],
  'actual_missing': ['repetitions', 'recovery'],
  'passed': False},
 {'query': 'entraînement pyramide',
  'expected_level': 'low',
  'actual_level': 'very_low',
  'expected_generate': False,
  'actual_generate': False,
  'expected_missing': ['duration', 'repetitions', 'recovery'],
  'actual_missing': ['duration', 'repetitions', 'intensity', 'recovery'],
  'passed': False},
 {'query': 'faire du vélo',
  'expected_level': 'very_low',
  'actual_level': 'very_low',
  'expected_generate': False,
  'actual_generate': False,
  'expected_missing': ['duration', 'repetitions', 'intensity', '

In [4]:
# Test 2: Génération et validation XML
def test_xml_generation():
    """Teste la génération et validation des fichiers XML"""
    print("\n🧪 TEST 2: Génération XML")
    print("-" * 40)
    
    # Test avec un entraînement simple
    test_workout = {
        'id': 99,
        'zone': 'Test',
        'structure': {'reps': 2, 'duration': 5, 'recovery': 2},
        'intensity': 80
    }
    
    # Génération XML de test
    workout_file = ET.Element("workout_file")
    
    author = ET.SubElement(workout_file, "author")
    author.text = "Vekta Test"
    
    name = ET.SubElement(workout_file, "name")
    name.text = f"Test {test_workout['zone']}"
    
    description = ET.SubElement(workout_file, "description")
    description.text = f"Test corpus #{test_workout['id']}"
    
    sporttype = ET.SubElement(workout_file, "sportType")
    sporttype.text = "bike"
    
    tags = ET.SubElement(workout_file, "tags")
    
    workout = ET.SubElement(workout_file, "workout")
    
    # Structure: warmup + 2x(5min@80% + 2min@50%) + cooldown
    warmup = ET.SubElement(workout, "Warmup", Duration="600", PowerLow="0.65", PowerHigh="0.65")
    
    structure = test_workout['structure']
    for rep in range(structure['reps']):
        interval = ET.SubElement(workout, "SteadyState",
                               Duration=str(int(structure['duration'] * 60)),
                               Power=str(test_workout['intensity']/100))
        if rep < structure['reps'] - 1:
            recovery = ET.SubElement(workout, "SteadyState",
                                   Duration=str(int(structure['recovery'] * 60)),
                                   Power="0.50")
    
    cooldown = ET.SubElement(workout, "Cooldown", Duration="600", PowerLow="0.50", PowerHigh="0.50")
    
    # Sauvegarde
    xml_content = ET.tostring(workout_file, encoding='unicode')
    
    os.makedirs("test_workouts", exist_ok=True)
    test_file = "test_workouts/test_xml.zwo"
    
    from xml.dom import minidom
    dom = minidom.parseString(xml_content)
    pretty_xml = dom.toprettyxml(indent="  ")
    pretty_xml = '\n'.join([line for line in pretty_xml.split('\n') if line.strip()])
    
    with open(test_file, 'w', encoding='utf-8') as f:
        f.write(pretty_xml)
    
    # Validation par re-parsing
    try:
        tree = ET.parse(test_file)
        root = tree.getroot()
        
        # Vérifications
        required_elements = ['author', 'name', 'description', 'sportType', 'workout']
        missing_elements = [elem for elem in required_elements if root.find(elem) is None]
        
        workout_elem = root.find('workout')
        segments_count = len(list(workout_elem)) if workout_elem else 0
        
        # Calcul durée
        total_duration = 0
        if workout_elem:
            for segment in workout_elem:
                duration = int(segment.get('Duration', 0))
                total_duration += duration
        
        # Vérifications attendues
        expected_duration = 10 + (2 * 5) + (1 * 2) + 10  # 32min = 1920s
        expected_segments = 1 + 2 + 1 + 1  # warmup + 2 intervals + 1 recovery + cooldown = 5
        
        duration_match = total_duration == expected_duration * 60
        segments_match = segments_count == expected_segments
        structure_valid = len(missing_elements) == 0
        
        print(f"✅ Fichier généré: {test_file}")
        print(f"✅ Taille: {os.path.getsize(test_file)} bytes")
        print(f"✅ XML valide: {structure_valid}")
        print(f"✅ Durée: {total_duration//60}min (attendu: {expected_duration}min) {'✅' if duration_match else '❌'}")
        print(f"✅ Segments: {segments_count} (attendu: {expected_segments}) {'✅' if segments_match else '❌'}")
        
        all_tests_passed = duration_match and segments_match and structure_valid
        
        if all_tests_passed:
            print("✅ Tous les tests XML sont OK")
        else:
            print("❌ Certains tests XML ont échoué")
            if missing_elements:
                print(f"   Éléments manquants: {missing_elements}")
        
        return all_tests_passed
        
    except Exception as e:
        print(f"❌ Erreur validation XML: {e}")
        return False

test_xml_generation()



🧪 TEST 2: Génération XML
----------------------------------------
✅ Fichier généré: test_workouts/test_xml.zwo
✅ Taille: 483 bytes
✅ XML valide: True
✅ Durée: 32min (attendu: 32min) ✅
✅ Segments: 5 (attendu: 5) ✅
✅ Tous les tests XML sont OK


True

In [5]:
# Test 3: Test de régression - Vérification que les améliorations n'ont pas cassé les fonctionnalités de base
def test_regression():
    """Teste que les fonctionnalités de base marchent toujours"""
    print("\n🧪 TEST 3: Régression")
    print("-" * 40)
    
    regression_tests = [
        # Test: recherche corpus de base
        {
            'name': 'Recherche corpus tempo',
            'test': lambda: test_validator.search_corpus("3x 10 min tempo")[1] > 0.8,
            'description': 'Score confiance élevé pour requête tempo exacte'
        },
        {
            'name': 'Recherche corpus endurance',
            'test': lambda: test_validator.search_corpus("1h endurance")[1] > 0.6,
            'description': 'Score confiance modéré pour requête endurance'
        },
        {
            'name': 'Détection entité durée',
            'test': lambda: 'duration' not in test_validator.detect_missing_entities("10 min tempo"),
            'description': 'Détection correcte de la durée'
        },
        {
            'name': 'Détection entité répétitions',
            'test': lambda: 'repetitions' not in test_validator.detect_missing_entities("3x tempo"),
            'description': 'Détection correcte des répétitions'
        },
        {
            'name': 'Validation seuil élevé',
            'test': lambda: test_validator.validate_query("3x 10 min tempo")['is_valid'],
            'description': 'Validation positive pour confiance élevée'
        },
        {
            'name': 'Validation seuil faible',
            'test': lambda: not test_validator.validate_query("pyramide")['is_valid'],
            'description': 'Validation négative pour confiance faible'
        }
    ]
    
    passed_tests = 0
    total_tests = len(regression_tests)
    
    for test in regression_tests:
        try:
            result = test['test']()
            status = "✅ PASS" if result else "❌ FAIL"
            print(f"{status} | {test['name']:<25} | {test['description']}")
            if result:
                passed_tests += 1
        except Exception as e:
            print(f"❌ ERROR | {test['name']:<25} | Erreur: {e}")
    
    print(f"\n📊 Tests de régression: {passed_tests}/{total_tests} passés")
    
    if passed_tests == total_tests:
        print("✅ Aucune régression détectée")
        return True
    else:
        print("❌ Régressions détectées - Vérifiez les modifications")
        return False

test_regression()



🧪 TEST 3: Régression
----------------------------------------
✅ PASS | Recherche corpus tempo    | Score confiance élevé pour requête tempo exacte
❌ FAIL | Recherche corpus endurance | Score confiance modéré pour requête endurance
✅ PASS | Détection entité durée    | Détection correcte de la durée
✅ PASS | Détection entité répétitions | Détection correcte des répétitions
✅ PASS | Validation seuil élevé    | Validation positive pour confiance élevée
✅ PASS | Validation seuil faible   | Validation négative pour confiance faible

📊 Tests de régression: 5/6 passés
❌ Régressions détectées - Vérifiez les modifications


False

In [6]:
# Résumé des tests et rapport final
def generate_test_report():
    """Génère un rapport de test complet"""
    print("\n📋 RAPPORT DE TESTS FINAL")
    print("=" * 50)
    
    # Relancer tous les tests pour le rapport
    print("🔄 Exécution de tous les tests...")
    
    confidence_results = test_confidence_thresholds()
    xml_result = test_xml_generation()
    regression_result = test_regression()
    
    # Calcul des statistiques
    confidence_passed = sum(1 for r in confidence_results if r['passed'])
    confidence_total = len(confidence_results)
    
    all_tests = [
        ("Seuils de confiance", confidence_passed, confidence_total),
        ("Génération XML", 1 if xml_result else 0, 1),
        ("Tests de régression", 1 if regression_result else 0, 1)
    ]
    
    total_passed = sum(passed for _, passed, _ in all_tests)
    total_tests = sum(total for _, _, total in all_tests)
    
    print(f"\n📊 RÉSULTATS GLOBAUX:")
    print(f"   Tests passés: {total_passed}/{total_tests}")
    print(f"   Taux de réussite: {total_passed/total_tests:.1%}")
    
    print(f"\n📝 DÉTAIL PAR CATÉGORIE:")
    for category, passed, total in all_tests:
        status = "✅" if passed == total else "❌"
        print(f"   {status} {category}: {passed}/{total}")
    
    # Recommandations
    print(f"\n💡 RECOMMANDATIONS:")
    if total_passed == total_tests:
        print("   ✅ Pipeline validé - Prêt pour production")
        print("   ✅ Toutes les améliorations fonctionnent correctement")
        print("   ✅ Aucune régression détectée")
    else:
        print("   ⚠️  Certains tests ont échoué")
        print("   🔧 Vérifiez les modifications avant déploiement")
        print("   📋 Consultez les détails des échecs ci-dessus")
    
    # Fichiers générés
    print(f"\n📁 FICHIERS DE TEST GÉNÉRÉS:")
    if os.path.exists("test_workouts/test_xml.zwo"):
        print(f"   📄 test_workouts/test_xml.zwo ({os.path.getsize('test_workouts/test_xml.zwo')} bytes)")
    
    print(f"\n🎯 AMÉLIORATIONS VALIDÉES:")
    print("   ✅ Validation intelligente avec seuils de confiance")
    print("   ✅ Messages d'erreur précis avec entités manquantes")
    print("   ✅ Génération XML réelle de fichiers .zwo")
    print("   ✅ Vérification de fidélité corpus ↔ XML")
    
    return {
        'total_passed': total_passed,
        'total_tests': total_tests,
        'success_rate': total_passed/total_tests,
        'all_passed': total_passed == total_tests
    }

# Génération du rapport final
report = generate_test_report()



📋 RAPPORT DE TESTS FINAL
🔄 Exécution de tous les tests...
🧪 TEST 1: Seuils de confiance
----------------------------------------
❌ FAIL | 3x 10 min tempo avec 5min récu | 0.950 | high
❌ FAIL | 1h endurance aerobic           | 0.850 | medium
❌ FAIL | entraînement pyramide          | 0.400 | very_low
✅ PASS | faire du vélo                  | 0.100 | very_low
❌ FAIL | 5x30s sprint max               | 0.770 | medium

📊 Résultat: 1/5 tests passés
❌ Certains tests ont échoué
   ÉCHEC: 3x 10 min tempo avec 5min récup
      Niveau attendu: high, obtenu: high
      Génération attendue: True, obtenue: True
      Entités manquantes attendues: [], obtenues: ['recovery']
   ÉCHEC: 1h endurance aerobic
      Niveau attendu: medium, obtenu: medium
      Génération attendue: True, obtenue: True
      Entités manquantes attendues: [], obtenues: ['repetitions', 'recovery']
   ÉCHEC: entraînement pyramide
      Niveau attendu: low, obtenu: very_low
      Génération attendue: False, obtenue: False
      